In [21]:
import pandas as pd
import numpy as np


from core import constants
from core.utils import *

log = get_logger()

pd.set_option("display.float_format", "{:.2f}".format)

## RPGF 3 Data Check and Cleanup

In [22]:
df = pd.read_csv("data/dummy_data_rpgf3.csv")

In [23]:
display(df.sample(10))

,Address,Has voted,Has published,Published at,Created at,Updated at,Projects in ballot,Votes
743,Hpb4XXfdjj,True,False,NaN,2023-10-08 15:49:38,2023-10-27 00:55:42,0,[]
382,b8QMQ2aAPO,True,False,NaN,2023-11-22 07:12:27,2023-11-30 12:40:59,0,[]
472,C2fS2YOVj7,True,True,2023-11-21 08:57:35,2023-10-05 15:03:09,2023-11-02 01:05:50,17,"[{'amount': '527570', 'projectId': 'proj99'}, ..."
505,vrKTfFRDs4,True,False,NaN,2023-09-17 14:31:36,2023-10-07 08:53:30,0,[]
316,TnXjsNNhPr,False,False,NaN,2023-10-18 23:20:15,2023-11-15 02:56:49,0,[]
375,5qpQ90n0rk,True,True,2023-11-29 16:22:50,2023-09-30 13:47:57,2023-11-24 17:54:26,15,"[{'amount': '956769', 'projectId': 'proj42'}, ..."
431,JX0RCkiRsi,True,True,2023-11-14 08:05:49,2023-09-03 02:15:07,2023-10-26 07:03:05,11,"[{'amount': '968902', 'projectId': 'proj216'},..."
393,2ilHRtAN5V,True,False,NaN,2023-09-12 12:06:34,2023-09-26 12:39:39,0,[]
915,JyulDg4OcV,False,False,NaN,2023-10-14 23:29:17,2023-10-21 09:20:43,0,[]
820,1JgeZknT31,True,False,NaN,2023-11-19 02:19:30,2023-11-26 01:07:33,0,[]


In [24]:
log.info("Check - Num Ballots: " + str(df["Has published"].count()))
log.info("Check - Num Submissions (Published): " + str(df["Has published"].sum()))

# Check if voter_address is unique
if df["Address"].nunique() == df.shape[0]:
    log.info("Check - Address is unique.")
else:
    diff = df.shape[0] - df["Address"].nunique()
    log.info(f"Check - Address is not unique. There are {diff} duplicates.")

# Check if all voters have voted
if df[df["Has voted"] == False].shape[0] > 0:
    not_voted = df[df["Has voted"] == False].shape[0]
    total = df["Address"].nunique()
    log.info(f"Check - {not_voted} voters out of {total} have not voted.")
else:
    log.info("Check - All voters have voted.")

# Check if all voters have published
if df[df["Has published"] == False].shape[0] > 0:
    not_voted = df[df["Has published"] == False].shape[0]
    total = df["Address"].nunique()
    log.info(f"Check - {not_voted} voters out of {total} have not published.")
else:
    log.info("Check - All voters have published.")

2024-01-02 23:12:51 INFO | Check - Num Ballots: 1000
2024-01-02 23:12:51 INFO | Check - Num Submissions (Published): 514
2024-01-02 23:12:51 INFO | Check - Address is unique.
2024-01-02 23:12:51 INFO | Check - 243 voters out of 1000 have not voted.
2024-01-02 23:12:51 INFO | Check - 486 voters out of 1000 have not published.


In [25]:
# Apply the function and concatenate results
expanded_list = [
    expand_json(safe_json_loads(row), idx) for idx, row in df["Votes"].items()
]
expanded_df = pd.concat(expanded_list, ignore_index=True)

result_df = expanded_df.set_index("original_index").join(df.set_index(df.index))

In [26]:
testing_address = "zgdSu8Yr87"
print_df = result_df[result_df["Address"] == testing_address]
print("Num Projects Voted : " + str(print_df["projectId"].count()))
display(print_df.head(10))

Num Projects Voted : 18


,amount,projectId,Address,Has voted,Has published,Published at,Created at,Updated at,Projects in ballot,Votes
830,703749,proj117,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,385034,proj51,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,470511,proj230,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,113619,proj204,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,156327,proj141,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,622540,proj390,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,32962,proj482,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,911617,proj46,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,755757,proj250,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,114436,proj82,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."


In [27]:
columns = [col for col in result_df.columns if col not in ["amount", "projectId"]]
columns += ["amount", "projectId"]  # Add the columns to the end of the list
result_df = result_df[columns]

# Update df columns names
result_df.columns = [
    "voter_address",
    "has_voted",
    "has_published",
    "published_at",
    "created_at",
    "updated_at",
    "projects_in_ballot",
    "votes",
    "amount",
    "project_id",
]

result_df.drop(columns="votes", inplace=True)

result_df["amount"] = pd.to_numeric(result_df["amount"])

In [28]:
# result_df.head()
result_df[result_df["voter_address"] == testing_address].head(70)

,voter_address,has_voted,has_published,published_at,created_at,updated_at,projects_in_ballot,amount,project_id
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,703749,proj117
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,385034,proj51
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,470511,proj230
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,113619,proj204
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,156327,proj141
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,622540,proj390
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,32962,proj482
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,911617,proj46
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,755757,proj250
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,114436,proj82


## Calculate Voting Results

In [29]:
allocator = ProjectAllocator(
    total_amount=constants.TOTAL_AMOUNT,
    min_amount=constants.MIN_AMOUNT,
    quorum=constants.QUORUM,
)

In [30]:
initial_allocation = allocator.calculate_initial_allocation(result_df)

In [31]:
display(initial_allocation.sample(10))

,votes_count,median_amount,is_eligible
project_id,,,
proj216,14,537007.00,False
proj236,15,417226.00,False
proj376,15,229330.00,False
proj168,5,747216.00,False
proj480,12,558717.50,False
proj276,15,440328.00,False
proj304,13,654721.00,False
proj399,12,658719.50,False
proj350,12,415465.00,False


In [32]:
# Scaling the total to 30M OP by project and filter out those with < 1500 OP
allocation_iter = initial_allocation[initial_allocation["is_eligible"] == True].copy()
allocation_iter["scaled_amount"] = allocation_iter["median_amount"]
# display(allocation_iter)
# Set a maximum number of iterations to prevent infinite loop
max_iterations = 10
current_iteration = 0

while (
    allocation_iter["scaled_amount"].sum() != constants.TOTAL_AMOUNT
    and current_iteration <= max_iterations
):
    allocation_iter = allocator.scale_allocations_oneby(allocation_iter)
    current_iteration += 1

    log.info("Check - Current iteration: " + str(current_iteration))

2024-01-02 23:12:51 INFO | Check - Original Amount Eligible: 12322787.0
2024-01-02 23:12:51 INFO | Check - Scale Factor: 2.434514205268662
2024-01-02 23:12:51 INFO | Check - No projects below minimum OP
2024-01-02 23:12:51 INFO | Check - Current iteration: 1


In [33]:
# Check if the loop exited due to reaching max iterations
if (
    current_iteration == max_iterations
    and allocation_iter["scaled_amount"].sum() != constants.TOTAL_AMOUNT
):
    log.info("Maximum iterations reached without meeting the total amount condition.")
else:
    final_total = allocation_iter["scaled_amount"].sum()
    log.info(
        f"Condition met with {final_total} OP allocated through {current_iteration} iteration(s)."
    )

2024-01-02 23:12:51 INFO | Condition met with 30000000.0 OP allocated through 1 iteration(s).


In [34]:
# join the initial allocation with the final allocation, if scaled_amount is null then make it 0
final_allocation = initial_allocation.merge(
    allocation_iter["scaled_amount"],
    how="left",
    on="project_id",
).fillna({"scaled_amount": 0})

# check if the final allocation table still contains all projects.
if final_allocation.index.nunique() == result_df["project_id"].nunique():
    log.info("Check - Final allocation table has included all the projects.")
else:
    log.info(
        "Check - Final allocation table has missing projects. Printing out the missing projects below."
    )
    log.info(
        result_df[~result_df["project_id"].isin(final_allocation.index)]["project_id"]
    )

# check if the final allocation table still sums to the total amount.
if final_allocation["scaled_amount"].sum() == final_total:
    log.info(
        "Check - Final allocation table sums to the right amount of OP: "
        + str(final_total)
    )
else:
    log.info(
        "Check - Final allocation table does not sum to the total OP. Printing out the missing amount below."
    )
    log.info(str(final_total - final_allocation["scaled_amount"].sum()) + " OP")

2024-01-02 23:12:51 INFO | Check - Final allocation table has included all the projects.
2024-01-02 23:12:51 INFO | Check - Final allocation table sums to the right amount of OP: 30000000.0


In [35]:
# export csv
# allocation_iter.drop(columns="median_amount", inplace=True)
final_allocation.to_csv("data/rpgf3_allocation_final.csv")

log.info(f"Results saved in data/rpgf3_allocation_final.csv.")

2024-01-02 23:12:51 INFO | Results saved in data/rpgf3_allocation_final.csv.


In [36]:
final_allocation.head(10)

,votes_count,median_amount,is_eligible,scaled_amount
project_id,,,,
proj26,5,841223.00,False,0.00
proj323,11,838886.00,False,0.00
proj123,7,829909.00,False,0.00
proj170,10,817377.00,False,0.00
proj46,11,802627.00,False,0.00
proj269,10,793657.00,False,0.00
proj12,11,787796.00,False,0.00
proj196,7,782668.00,False,0.00
proj287,6,782357.00,False,0.00


In [37]:
to_cut = (
    allocation_iter[allocation_iter["scaled_amount"] < 1500]
    .sort_values(by="scaled_amount")
    .head(1)
)

In [38]:
# check if to_cut is empty
to_cut.empty

True

### Calculate Voting Results using pytorch

In [39]:

result_tensor, num_projects = allocator.convert_df_to_tensor(result_df)
project_tensors = allocator.get_project_tensor(result_tensor, num_projects)


In [40]:
# export to onnx
allocator.eval()

# convert projects tensors to tuple
initial_allocation = allocator.forward(*project_tensors)
initial_allocation.shape

input_names = ['input_' + str(i) for i in range(len(project_tensors))]
    # Export the model
torch.onnx.export(allocator,               # model being run
                      tuple(project_tensors),          # model input (or a tuple for multiple inputs)
                      "network.onnx",           # where to save the model (can be a file or file-like object)
                      export_params=False,       # store the trained parameter weights inside the model file
                      opset_version=17,         # the ONNX version to export the model to
                      do_constant_folding=True, # whether to execute constant folding for optimization
                      input_names = input_names,   # the model's input names
                      output_names = ['output'])

/Users/alexandercamuto/Documents/GitHub/zk-rpgf/core/utils.py:164: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  votes_count = torch.tensor([num_bids]).reshape(1, 1)


/Users/alexandercamuto/Documents/GitHub/zk-rpgf/.env/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [41]:
import ezkl
import os

model_path = "network.onnx"
settings_path = "settings.json"
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "public"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # private by default

res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
assert res == True

In [42]:
data_path = os.path.join("input.json")

data = dict(input_data = [tensor.detach().numpy().reshape([-1]).tolist() for tensor in project_tensors])

# Serialize data into file:
json.dump(data, open(data_path, 'w'))


ezkl.calibrate_settings(data_path, model_path, settings_path, "resources")


In [ ]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [ ]:
# srs path
res = ezkl.get_srs( settings_path)

In [ ]:
# now generate the witness file 

res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [ ]:

# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK



res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [ ]:
# GENERATE A PROOF


proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        "single",
    )

print(res)
assert os.path.isfile(proof_path)

In [ ]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
    )

assert res == True
print("verified")